In [1]:
# a) Data Preparation

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
print("Imports complete!\n")

# Demo text
raw_text = """Machine learning enables computers to discover patterns from data.
Neural networks are powerful models that learn representations automatically.
Natural language processing allows machines to understand human communication.
Deep learning methods are transforming artificial intelligence research and applications.""".split()

vocab = sorted(set(raw_text))
vocab_size = len(vocab)
word_to_ix = {word: ix for ix, word in enumerate(vocab)}
ix_to_word = {ix: word for word, ix in word_to_ix.items()}

print("Vocabulary size:", vocab_size)
print("First 5 word-index pairs:", list(word_to_ix.items())[:5])


Imports complete!

Vocabulary size: 34
First 5 word-index pairs: [('Deep', 0), ('Machine', 1), ('Natural', 2), ('Neural', 3), ('allows', 4)]


In [2]:
# b) Generate Training Data

CONTEXT_SIZE = 2         # Two words before, two after
EMBEDDING_DIM = 50       # Size of word embedding vectors

def make_context_vector(context, word_to_ix):
    return [word_to_ix[w] for w in context]

data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = [
        raw_text[i - 2], raw_text[i - 1],
        raw_text[i + 1], raw_text[i + 2]
    ]
    target = raw_text[i]
    data.append((context, target))

X = np.array([make_context_vector(ctx, word_to_ix) for ctx, _ in data])
y = np.array([word_to_ix[target] for _, target in data])

print("Shape of X (contexts):", X.shape)
print("Shape of y (targets):", y.shape)
print("Sample context/target:\n  Context:", [vocab[ix] for ix in X[0]], "\n  Target:", vocab[y[0]])

Shape of X (contexts): (33, 4)
Shape of y (targets): (33,)
Sample context/target:
  Context: ['Machine', 'learning', 'computers', 'to'] 
  Target: enables


In [3]:
# c) Define CBOW model in Keras

inputs = layers.Input(shape=(4,), dtype="int32")  # 4 context words (indices)
embed = layers.Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM)(inputs)
avg_embed = layers.Lambda(lambda x: tf.reduce_mean(x, axis=1))(embed)
dense1 = layers.Dense(128, activation="relu")(avg_embed)
outputs = layers.Dense(vocab_size, activation="softmax")(dense1)

cbow_model = models.Model(inputs=inputs, outputs=outputs)
cbow_model.compile(
    optimizer=optimizers.Adam(learning_rate=0.01),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
print("CBOW model summary:")
cbow_model.summary()


CBOW model summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 4)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 4, 50)               │           1,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           6,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 34)                  │           4,386 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,614 (49.27 KB)

 Trainable params: 12,614 (49.27 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# d) Train the model

history = cbow_model.fit(
    X, y,
    epochs=100,
    batch_size=8,
    verbose=1
)
print("Training done!\n")

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0000e+00 - loss: 3.5421  
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1717 - loss: 3.4669 
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2524 - loss: 3.3259 
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3542 - loss: 3.0758 
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3389 - loss: 2.6599 
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4129 - loss: 2.1166 
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5843 - loss: 1.6756 
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6282 - loss: 1.1565 
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8450 - loss: 0.7872 
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9676 - loss: 0.5244 
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9673 - loss: 0.3197 
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9777 

In [5]:
# e) Output: Test context for prediction

test_context = ['Neural', 'networks', 'that', 'learn']
test_input = np.array([make_context_vector(test_context, word_to_ix)])
pred_probs = cbow_model.predict(test_input)
pred_idx = np.argmax(pred_probs[0])
pred_word = ix_to_word[pred_idx]

print("\n--------------------------------------------")
print("Test Context Words:", test_context)
print("Predicted Target Word:", pred_word)
print("--------------------------------------------")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step

--------------------------------------------
Test Context Words: ['Neural', 'networks', 'that', 'learn']
Predicted Target Word: models
--------------------------------------------
